# POULTRY BIOGAS MODEL TRAINING AND SAVING SCRIPT


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
import warnings
warnings.filterwarnings("ignore")

from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import joblib
import json
import os
from datetime import datetime

# POULTRY BIOGAS MODEL CLASS WITH SAVE/LOAD FUNCTIONALITY


In [ ]:


class PoultryBiogasModel:
    """Clean Poultry-Specific Biogas Prediction Model with Save/Load functionality"""
    
    def __init__(self):
        self.model = None
        self.scaler = RobustScaler()
        self.feature_names = []
        self.is_trained = False
        self.model_performance = {}
        self.training_data = None
        self.model_metadata = {}
    
    def load_and_filter_poultry_data(self, df):
        """Load data and filter for poultry operations only"""
        # Filter for poultry operations only
        poultry_mask = df['Poultry'] > 0
        
        # Also check Animal/Farm Type for poultry keywords if column exists
        if 'Animal/Farm Type(s)' in df.columns:
            poultry_keywords = ['Poultry', 'Chicken', 'Broiler', 'Layer', 'Turkey', 'Duck', 'Egg']
            type_mask = df['Animal/Farm Type(s)'].str.contains(
                '|'.join(poultry_keywords), case=False, na=False
            )
            df_poultry = df[poultry_mask | type_mask].copy()
        else:
            df_poultry = df[poultry_mask].copy()
        
        # Set other animal types to zero for poultry-only model
        animal_cols_to_zero = ['Cattle', 'Dairy', 'Swine']
        for col in animal_cols_to_zero:
            if col in df_poultry.columns:
                df_poultry[col] = 0
        
        return df_poultry
    
    def create_poultry_features(self, df):
        """Create features specific to poultry biogas production"""
        # Daily waste production (0.11 kg per bird per day)
        df['Poultry_Daily_Waste_kg'] = df['Poultry'] * 0.11
        
        # Expected biogas potential (0.45 m³/kg, converted to cu-ft)
        df['Expected_Biogas_Potential'] = df['Poultry_Daily_Waste_kg'] * 0.45 * 35.315
        
        # Operation scale categories
        df['Poultry_Scale'] = pd.cut(df['Poultry'], 
                                   bins=[0, 10000, 50000, 200000, float('inf')],
                                   labels=['Small', 'Medium', 'Large', 'Industrial'],
                                   right=False)
        
        # Calculate operational years if not present
        if 'Operational Years' not in df.columns and 'Year Operational' in df.columns:
            current_year = 2024
            df['Operational Years'] = current_year - df['Year Operational']
            df['Operational Years'] = np.maximum(df['Operational Years'], 1)
        
        # Ensure Total_Animals equals Poultry
        df['Total_Animals'] = df['Poultry']
        
        return df
    
    def preprocess_data(self, df):
        """Clean preprocessing with minimal output"""
        # Drop location columns
        location_cols = ['Project Name', 'City', 'County', 'State']
        df = df.drop([col for col in location_cols if col in df.columns], axis=1)
        
        # Create poultry features
        df = self.create_poultry_features(df)
        
        # Handle missing values
        numeric_cols = df.select_dtypes(include=[np.number]).columns
        for col in numeric_cols:
            if df[col].isnull().sum() > 0:
                df[col].fillna(df[col].median(), inplace=True)
        
        categorical_cols = df.select_dtypes(include=['object']).columns
        for col in categorical_cols:
            if df[col].isnull().sum() > 0:
                mode_val = df[col].mode().iloc[0] if len(df[col].mode()) > 0 else 'Unknown'
                df[col].fillna(mode_val, inplace=True)
        
        # Remove outliers
        numeric_cols = df.select_dtypes(include=[np.number]).columns
        target_col = 'Biogas Generation Estimate (cu-ft/day)'
        numeric_cols = [col for col in numeric_cols if col != target_col]
        
        for col in numeric_cols:
            if df[col].nunique() > 10:
                Q1, Q3 = df[col].quantile([0.05, 0.95])
                IQR = Q3 - Q1
                if IQR > 0:
                    lower_bound = Q1 - 1.5 * IQR
                    upper_bound = Q3 + 1.5 * IQR
                    df[col] = np.clip(df[col], lower_bound, upper_bound)
        
        return df
    
    def train_model(self, df):
        """Train model with clean output"""
        print("Starting model training...")
        
        # Filter and preprocess
        df_poultry = self.load_and_filter_poultry_data(df)
        df_processed = self.preprocess_data(df_poultry)
        self.training_data = df_processed.copy()
        
        print(f"Training on {len(df_poultry)} poultry operations")
        
        # Prepare features and target
        target_col = 'Biogas Generation Estimate (cu-ft/day)'
        if target_col not in df_processed.columns:
            raise ValueError(f"Target column '{target_col}' not found")
        
        y = df_processed[target_col]
        X = df_processed.drop(columns=[target_col])
        
        # Handle categorical variables
        categorical_cols = X.select_dtypes(include=['object', 'category']).columns
        if len(categorical_cols) > 0:
            X_encoded = pd.get_dummies(X, columns=categorical_cols, drop_first=True)
        else:
            X_encoded = X.copy()
        
        # Remove non-numeric columns
        non_numeric_cols = X_encoded.select_dtypes(exclude=[np.number]).columns
        if len(non_numeric_cols) > 0:
            X_encoded = X_encoded.drop(columns=non_numeric_cols)
        
        self.feature_names = X_encoded.columns.tolist()
        
        # Scale and split
        X_scaled = self.scaler.fit_transform(X_encoded)
        X_train, X_test, y_train, y_test = train_test_split(
            X_scaled, y, test_size=0.2, random_state=42
        )
        
        # Train models
        models = {
            'Random Forest': RandomForestRegressor(n_estimators=100, max_depth=12, random_state=42),
            'Gradient Boosting': GradientBoostingRegressor(n_estimators=100, random_state=42),
            'Ridge Regression': Ridge(alpha=10.0),
            'Linear Regression': LinearRegression()
        }
        
        best_model = None
        best_score = float('inf')
        results = []
        
        print("Training different models...")
        for name, model in models.items():
            try:
                print(f"Training {name}...")
                model.fit(X_train, y_train)
                y_pred = model.predict(X_test)
                
                rmse = np.sqrt(mean_squared_error(y_test, y_pred))
                r2 = r2_score(y_test, y_pred)
                mae = mean_absolute_error(y_test, y_pred)
                
                results.append({
                    'Model': name,
                    'RMSE': rmse,
                    'R²': r2,
                    'MAE': mae
                })
                
                print(f"{name} - RMSE: {rmse:.2f}, R²: {r2:.3f}")
                
                if rmse < best_score:
                    best_score = rmse
                    best_model = model
            except Exception as e:
                print(f"Error training {name}: {str(e)}")
                continue
        
        if best_model is not None:
            self.model = best_model
            self.is_trained = True
            
            if results:
                results_df = pd.DataFrame(results).sort_values('RMSE')
                self.model_performance = results_df.iloc[0].to_dict()
                
                # Store metadata
                self.model_metadata = {
                    'training_date': datetime.now().isoformat(),
                    'training_samples': len(df_poultry),
                    'feature_count': len(self.feature_names),
                    'best_model_name': self.model_performance['Model'],
                    'performance_metrics': self.model_performance
                }
                
                print(f"\nBest model: {self.model_performance['Model']}")
                print(f"RMSE: {self.model_performance['RMSE']:.2f}")
                print(f"R² Score: {self.model_performance['R²']:.3f}")
        
        return pd.DataFrame(results), len(df_poultry)
    
    def save_model(self, model_dir='saved_models'):
        """Save the trained model and all related components"""
        if not self.is_trained:
            raise ValueError("No trained model to save!")
        
        # Create directory if it doesn't exist
        os.makedirs(model_dir, exist_ok=True)
        
        # Generate timestamp for unique filename
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        
        # Save the trained model
        model_path = os.path.join(model_dir, f'poultry_biogas_model_{timestamp}.joblib')
        joblib.dump(self.model, model_path)
        
        # Save the scaler
        scaler_path = os.path.join(model_dir, f'poultry_biogas_scaler_{timestamp}.joblib')
        joblib.dump(self.scaler, scaler_path)
        
        # Save feature names and metadata
        metadata = {
            'feature_names': self.feature_names,
            'model_performance': self.model_performance,
            'model_metadata': self.model_metadata,
            'model_file': f'poultry_biogas_model_{timestamp}.joblib',
            'scaler_file': f'poultry_biogas_scaler_{timestamp}.joblib'
        }
        
        metadata_path = os.path.join(model_dir, f'poultry_biogas_metadata_{timestamp}.json')
        with open(metadata_path, 'w') as f:
            json.dump(metadata, f, indent=2)
        
        # Also save as 'latest' for easy loading
        latest_model_path = os.path.join(model_dir, 'latest_poultry_biogas_model.joblib')
        latest_scaler_path = os.path.join(model_dir, 'latest_poultry_biogas_scaler.joblib')
        latest_metadata_path = os.path.join(model_dir, 'latest_poultry_biogas_metadata.json')
        
        joblib.dump(self.model, latest_model_path)
        joblib.dump(self.scaler, latest_scaler_path)
        with open(latest_metadata_path, 'w') as f:
            json.dump(metadata, f, indent=2)
        
        print(f"Model saved successfully!")
        print(f"Model file: {model_path}")
        print(f"Scaler file: {scaler_path}")
        print(f"Metadata file: {metadata_path}")
        print(f"Latest files also saved for easy loading")
        
        return {
            'model_path': model_path,
            'scaler_path': scaler_path,
            'metadata_path': metadata_path
        }
    
    def load_model(self, model_dir='saved_models', use_latest=True):
        """Load a saved model"""
        if use_latest:
            model_path = os.path.join(model_dir, 'latest_poultry_biogas_model.joblib')
            scaler_path = os.path.join(model_dir, 'latest_poultry_biogas_scaler.joblib')
            metadata_path = os.path.join(model_dir, 'latest_poultry_biogas_metadata.json')
        else:
            # For specific timestamp, you would need to specify the files
            raise NotImplementedError("Specify timestamp-based loading if needed")
        
        try:
            # Load model
            self.model = joblib.load(model_path)
            
            # Load scaler
            self.scaler = joblib.load(scaler_path)
            
            # Load metadata
            with open(metadata_path, 'r') as f:
                metadata = json.load(f)
            
            self.feature_names = metadata['feature_names']
            self.model_performance = metadata['model_performance']
            self.model_metadata = metadata['model_metadata']
            self.is_trained = True
            
            print("Model loaded successfully!")
            print(f"Model: {self.model_performance.get('Model', 'Unknown')}")
            print(f"R² Score: {self.model_performance.get('R²', 0):.3f}")
            print(f"Training date: {self.model_metadata.get('training_date', 'Unknown')}")
            
            return True
            
        except Exception as e:
            print(f"Error loading model: {str(e)}")
            return False
    
    def predict_biogas(self, poultry_count, digester_type='Complete Mix', 
                      co_digestion='No', operational_years=5, **kwargs):
        """Make prediction with clean output"""
        if not self.is_trained:
            raise ValueError("Model not trained!")
        
        # Create input data
        input_data = pd.DataFrame({
            'Poultry': [poultry_count],
            'Digester Type': [digester_type],
            'Co-Digestion': [co_digestion],
            'Operational Years': [operational_years],
            'Project Type': [kwargs.get('project_type', 'New')],
            'Status': [kwargs.get('status', 'Operational')],
            'Year Operational': [2024 - operational_years],
            'Biogas End Use(s)': [kwargs.get('biogas_end_use', 'Electricity')],
            'LCFS Pathway?': [kwargs.get('lcfs_pathway', 'No')],
            'Awarded USDA Funding?': [kwargs.get('usda_funding', 'No')],
            'Cattle': [0],
            'Dairy': [0],
            'Swine': [0],
            'Electricity Generated (kWh/yr)': [kwargs.get('electricity_kwh', 0)],
            'Total Emission Reductions (MTCO2e/yr)': [kwargs.get('emissions', 0)]
        })
        
        # Process input
        input_data = self.create_poultry_features(input_data)
        target_col = 'Biogas Generation Estimate (cu-ft/day)'
        if target_col in input_data.columns:
            input_data = input_data.drop(columns=[target_col])
        
        # Encode and prepare
        categorical_cols = input_data.select_dtypes(include=['object', 'category']).columns
        if len(categorical_cols) > 0:
            input_encoded = pd.get_dummies(input_data, columns=categorical_cols, drop_first=True)
        else:
            input_encoded = input_data.copy()
        
        # Align features
        for col in self.feature_names:
            if col not in input_encoded.columns:
                input_encoded[col] = 0
        
        input_encoded = input_encoded[self.feature_names]
        input_scaled = self.scaler.transform(input_encoded)
        
        # Predict
        predicted_biogas = self.model.predict(input_scaled)[0]
        
        # Calculate metrics
        return self.calculate_metrics(poultry_count, predicted_biogas)
    
    def calculate_metrics(self, poultry_count, predicted_biogas):
        """Calculate comprehensive metrics"""
        daily_waste_kg = poultry_count * 0.11
        annual_biogas_cuft = predicted_biogas * 365
        
        # Gas composition
        methane_content = 0.60
        daily_methane_cuft = predicted_biogas * methane_content
        daily_co2_cuft = predicted_biogas * 0.35
        
        # Energy calculations
        daily_energy_btu = predicted_biogas * 600
        annual_energy_mmbtu = daily_energy_btu * 365 / 1_000_000
        
        # Economics
        energy_price_per_mmbtu = 5.0
        annual_energy_value = annual_energy_mmbtu * energy_price_per_mmbtu
        
        # Environmental
        daily_methane_kg = daily_methane_cuft * 0.0007168
        annual_co2_equivalent_avoided = daily_methane_kg * 365 * 28 / 1000
        
        # Efficiency
        theoretical_max_biogas = daily_waste_kg * 0.45 * 35.315
        system_efficiency = (predicted_biogas / theoretical_max_biogas * 100) if theoretical_max_biogas > 0 else 0
        
        return {
            'daily_biogas_cuft': round(predicted_biogas, 0),
            'annual_biogas_cuft': round(annual_biogas_cuft, 0),
            'daily_methane_cuft': round(daily_methane_cuft, 0),
            'daily_co2_cuft': round(daily_co2_cuft, 0),
            'daily_waste_kg': round(daily_waste_kg, 1),
            'annual_waste_kg': round(daily_waste_kg * 365, 0),
            'daily_energy_btu': round(daily_energy_btu, 0),
            'annual_energy_mmbtu': round(annual_energy_mmbtu, 1),
            'annual_energy_value_usd': round(annual_energy_value, 0),
            'annual_co2_equivalent_avoided_tonnes': round(annual_co2_equivalent_avoided, 1),
            'system_efficiency_percent': round(system_efficiency, 1),
            'biogas_per_bird_cuft_day': round(predicted_biogas / poultry_count, 3),
            'waste_per_bird_kg_day': round(0.11, 3),
            'theoretical_max_cuft_day': round(theoretical_max_biogas, 0)
        }


# TRAINING SCRIPT MAIN FUNCTION


In [ ]:

def train_and_save_model(csv_file_path):
    """Train the model and save it"""
    try:
        # Load data
        print(f"Loading data from {csv_file_path}...")
        df = pd.read_csv(csv_file_path)
        print(f"Loaded dataset with {len(df)} rows and {len(df.columns)} columns")
        
        # Initialize model
        model = PoultryBiogasModel()
        
        # Train model
        results_df, poultry_count = model.train_model(df)
        
        # Save model
        save_paths = model.save_model()
        
        print("\n" + "="*50)
        print("TRAINING COMPLETED SUCCESSFULLY!")
        print("="*50)
        print(f"Training samples: {poultry_count}")
        print(f"Best amodel: {model.model_performance['Model']}")
        print(f"RMSE: {model.model_performance['RMSE']:.2f}")
        print(f"R² Score: {model.model_performance['R²']:.3f}")
        print("\nModel files saved:")
        for key, path in save_paths.items():
            print(f"- {key}: {path}")
        
        return model, results_df
        
    except Exception as e:
        print(f"Error during training: {str(e)}")
        return None, None

if __name__ == "__main__":
    # Example usage
    print("Poultry Biogas Model Training Script")
    print("="*40)
    csv_file= "biogas_data.csv"
    if os.path.exists(csv_file):
        model, results = train_and_save_model(csv_file)
        if model:
            print("\nTraining completed! You can now use the analysis app.")
    else:
        print(f"File {csv_file} not found!")
        print("Please ensure your CSV file exists and contains the required columns.")